# **NRMSE Frame Similarity Assessment - Detailed Process Explanation**

# **Step 1: Import Libraries**
# First, we import the required libraries for video processing and memory profiling:
# - `cv2` (OpenCV): This library provides functions to read and process video frames.
# - `numpy`: Used for numerical operations, especially to compute frame differences and error metrics like NRMSE.
# - `time`: This helps to track how long each part of the process takes, providing useful time metrics.
# - `psutil`: This library allows us to track the memory usage of the program, which is useful for profiling.

# **Step 2: Set the Video Paths**
# We define the paths for the input video and the output processed video:
# - `video_path` points to the location of the video we want to process.
# - `output_video_path` is where we save the processed video after performing the frame similarity analysis.

# **Step 3: NRMSE Frame Similarity Assessment Function**
# The core of the frame similarity is the NRMSE (Normalized Root Mean Square Error) calculation:
# - NRMSE measures the difference between two frames, which is useful for quantifying their dissimilarity.
# - The function first computes the squared differences between corresponding pixels in the two frames.
# - Then, the mean squared error (MSE) is computed by averaging the squared differences.
# - NRMSE is calculated by taking the square root of the MSE and normalizing it by the pixel intensity range (difference between the maximum and minimum pixel values).
# - This results in a value between 0 and 1, where 0 indicates identical frames and higher values indicate more dissimilarity.

# **Step 4: Define the Frame Similarity Function**
# This function (`is_frame_similar`) is used to decide whether two frames are similar enough based on their NRMSE:
# - The function calls `calculate_nrmse` to get the NRMSE between the two frames.
# - The NRMSE is then converted to a similarity score by subtracting it from 1. This way, lower NRMSE values (indicating more similarity) correspond to higher similarity scores.
# - If the similarity score is greater than or equal to a predefined threshold (default 0.82), the frames are considered similar and the frame is skipped. Otherwise, it will be included in the processed video.

# **Step 5: Load Video Frames**
# We use OpenCV to load all frames from the video:
# - The video is read frame by frame using `cv2.VideoCapture`.
# - Each frame is then converted to grayscale using `cv2.cvtColor`. Grayscale frames are used to simplify the computation by reducing the complexity of color channels (this helps in focusing on structural changes between frames).
# - All the frames are stored in a list, which will later be processed to compare similarity.

# **Step 6: Load Video Frames and Measure Loading Time**
# The frames are read into memory, and the time taken to load the video frames is measured:
# - `get_video_frames` function is called, and the elapsed time is recorded.
# - The frames are loaded one by one, and we can track how much time it takes to load the entire video into memory.

# **Step 7: Set Up Video Writer**
# To save the processed frames into a new video, we need to set up a video writer:
# - `cv2.VideoWriter_fourcc` is used to specify the codec used for saving the video. Here, the `mp4v` codec is selected.
# - The frames per second (fps) for the output video is set to match the original video’s fps to maintain playback consistency.
# - The resolution (height and width) of the video is determined from the first frame, as all frames in the video will have the same size.
# - This setup allows us to write the processed frames to the output file.

# **Step 8: Process Video Frames and Write to Output**
# Now, we begin processing the frames:
# - The first frame is always written to the output video, as it serves as the baseline for comparison.
# - For each subsequent frame, the function compares it with the last processed frame using the `is_frame_similar` function.
# - If the frames are deemed similar (i.e., their NRMSE is below the threshold), the frame is skipped, and we move on to the next frame.
# - If the frames are different enough, the current frame is written to the output video, and the index of the last processed frame is updated.
# - This process reduces the size of the output video by removing frames that are too similar to the preceding one.

# **Step 9: Calculate Processing Time**
# After processing all frames, we calculate the time taken for frame comparison and writing the output:
# - `processing_time` represents the time it took to compare frames and write the processed frames to the output video.
# - `total_time` is the sum of the time taken to load the frames and process them.
# - These time metrics are useful for performance evaluation, helping us understand how long it takes to process the video and which steps might need optimization.

# **Step 10: Release Video Writer**
# After processing all the frames, we release the video writer:
# - `out.release()` is called to finalize and save the processed video to the specified output path.
# - This step is essential for closing the video file and making sure all the frames are written correctly.

# **Step 11: Memory Profiling**
# To track the memory consumption during the video processing:
# - We use `psutil` to measure the memory usage before and after processing.
# - `memory_info.rss` gives the current memory used by the process, and we calculate the difference between the initial and final memory usage.
# - We also estimate the final memory consumption by considering the number of skipped frames, as each skipped frame would have been stored in memory but not written to the output.
# - Memory profiling is important for understanding the efficiency of the program, especially when processing large videos.

# **Step 12: Log the Results**
# Finally, we print out useful metrics:
# - `Processed video saved as`: Displays the path of the output processed video.
# - `Total number of frames`: Shows the total number of frames in the video.
# - `Number of frames skipped`: Shows how many frames were skipped because they were too similar to the previous frame.
# - `Percentage of frames skipped`: Provides the percentage of frames that were skipped.
# - `Frames processing per second`: Indicates how fast the frames were processed (how many frames were processed per second).
# - Time metrics: Including the time taken to load frames, process them, and the total processing time.
# - Memory usage: Shows the initial and final memory consumption, as well as the memory consumed during processing.
# - Total processing load: Indicates how many frames were processed, giving insight into the scale of the task.



In [5]:
import cv2
import numpy as np
import time
import psutil  # For memory profiling

# Set the video path (update this to your video file path)
video_path = '/content/00067cfb-e535423e.mov'  # Change this to your video file path
output_video_path = '/content/processed_video_nrmse.mp4'  # Output video path

# Function to calculate Normalized Root Mean Square Error (NRMSE) between frames
def calculate_nrmse(frame1, frame2):
    # Compute the squared difference between the frames
    diff = (frame1.astype(np.float32) - frame2.astype(np.float32)) ** 2

    # Calculate the mean squared error (MSE)
    mse = np.mean(diff)

    # Calculate the normalized root mean square error (NRMSE)
    max_pixel = np.max(frame1)
    min_pixel = np.min(frame1)

    nrmse = np.sqrt(mse) / (max_pixel - min_pixel)  # Normalize by the pixel intensity range
    return nrmse

# Function to check if frames are similar based on NRMSE similarity score
def is_frame_similar(frame1, frame2, threshold=0.82):
    # Calculate the NRMSE value between frames
    nrmse_value = calculate_nrmse(frame1, frame2)

    # Convert NRMSE to similarity score (1 - NRMSE)
    similarity_score = 1 - nrmse_value

    # Check if the similarity score is above the threshold (indicating similarity)
    return similarity_score >= threshold

# Read video frames
def get_video_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        frames.append(gray_frame)
    cap.release()
    return frames

# Load video frames
start_time = time.time()  # Start time for reading frames
frames = get_video_frames(video_path)
loading_time = time.time() - start_time  # Time taken to load frames

# Set up the video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for mp4
fps = 30  # Set to your video's frame rate
frame_height, frame_width = frames[0].shape
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Process frames and write to output video
frame_count = len(frames)
skipped_frames = 0
out.write(cv2.cvtColor(frames[0], cv2.COLOR_GRAY2BGR))  # Write the first frame
last_processed_frame_index = 0  # Track the last processed frame index

# Start time for processing
start_processing_time = time.time()

# Start comparing from frame 1 onwards
for i in range(1, frame_count):
    # Compare current frame with the last processed frame using NRMSE similarity score
    if is_frame_similar(frames[last_processed_frame_index], frames[i]):
        skipped_frames += 1
    else:
        out.write(cv2.cvtColor(frames[i], cv2.COLOR_GRAY2BGR))  # Write the current frame if not similar
        last_processed_frame_index = i  # Update the last processed frame index

# Calculate processing time
end_processing_time = time.time()
processing_time = end_processing_time - start_processing_time
total_time = loading_time + processing_time  # Total time includes loading and processing

# Release the video writer
out.release()

# Memory usage before and after processing
process = psutil.Process()
memory_info = process.memory_info()
initial_memory = memory_info.rss / (1024 * 1024)  # Convert bytes to MB
final_memory = (memory_info.rss + (skipped_frames * frames[0].nbytes)) / (1024 * 1024)  # Rough estimate of final memory
memory_consumed = final_memory - initial_memory

# Log the required information
print(f"Processed video saved as: {output_video_path}")
print(f"Total number of frames: {frame_count}")
print(f"Number of frames skipped: {skipped_frames}")
print(f"% of frames skipped: {(skipped_frames / frame_count) * 100:.2f}%")
print(f"Frames processing per second: {frame_count / processing_time:.2f} FPS")
print(f"Total time taken to load frames: {loading_time:.6f} seconds")
print(f"Total processing time: {processing_time:.6f} seconds")
print(f"Total time (loading + processing): {total_time:.6f} seconds")
print(f"Average time per frame comparison: {processing_time / (frame_count - 1):.6f} seconds")
print(f"Total memory usage: Initial = {initial_memory:.2f} MB, Final = {final_memory:.2f} MB")
print(f"Memory consumed during processing: {memory_consumed:.2f} MB")
print(f"Total processing load: {frame_count} frames")


Processed video saved as: /content/processed_video_nrmse.mp4
Total number of frames: 1206
Number of frames skipped: 1114
% of frames skipped: 92.37%
Frames processing per second: 383.64 FPS
Total time taken to load frames: 9.608807 seconds
Total processing time: 3.143576 seconds
Total time (loading + processing): 12.752383 seconds
Average time per frame comparison: 0.002609 seconds
Total memory usage: Initial = 2284.54 MB, Final = 3263.64 MB
Memory consumed during processing: 979.10 MB
Total processing load: 1206 frames
